In [1]:
import os, sys

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding, \
  Bidirectional, RepeatVector, Concatenate, Activation, Dot, Lambda
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K

import numpy as np
import matplotlib.pyplot as plt

if len(K.tensorflow_backend._get_available_gpus()) > 0:
    from keras.layers import CuDNNLSTM as LSTM
    from keras.layers import CuDNNGRU as GRU


Using TensorFlow backend.


- z = concat[ s(t-1),$h(t')$] 
- z = $tanh(W_1 z + b_1)$ # layer 1 of ANN
- z = softmax($W_2 z + b_2)$ # layer 2 of ANN
- Alphas must sum to 1 over time
$\sum_{t'=1}^{T_x} \alpha(t') = 1 $
- we use this special softmax over time
\begin{equation*}
\alpha(t') = \frac{exp(out(t'))}{\sum_{\tau=1}^{T_x} exp(out(\tau))}
\end{equation*}

In [33]:
# make sure we do softmax over the time axis
# expected shape is N x T x D(feature vector)
# note: the latest version of Keras allows you to pass in axis arg
def softmax_over_time(x):
    assert(K.ndim(x) > 2)
    e = K.exp(x - K.max(x, axis=1, keepdims=True))
    s = K.sum(e, axis=1, keepdims=True)
    return e / s


In [2]:
# config
BATCH_SIZE = 64
EPOCHS = 100
LATENT_DIM = 256
LATENT_DIM_DECODER = 256 # idea: make it different to ensure things all fit together properly!
NUM_SAMPLES = 10000
MAX_SEQUENCE_LENGTH = 100
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100

In [3]:
# Where we will store the data
input_texts = [] # sentence in original language
target_texts = [] # sentence in target language
target_texts_inputs = [] # sentence in target language offset by 1

In [4]:
# load in the data
# download the data at: http://www.manythings.org/anki/
count = 0
t = 0
for line in open('./large_files/spa-eng/spa.txt'):
    # only keep a limited number of samples
    t += 1
    if t > NUM_SAMPLES:
        break

    # input and target are separated by tab
    if '\t' not in line:
        continue

    # split up the input and translation
    input_text, translation = line.rstrip().split('\t')

    # make the target input and output
    # recall we'll be using teacher forcing
    target_text = translation + ' <eos>'
    target_text_input = '<sos> ' + translation

    input_texts.append(input_text)
    target_texts.append(target_text)
    target_texts_inputs.append(target_text_input)
    count += 1
    if count > 100:
        break
print("num samples:", len(input_texts))


num samples: 101


In [12]:
input_texts[0],target_texts[0]

('Go.', 'Ve. <eos>')

In [16]:
np_input_texts = np.array(input_texts)
np_input_texts.shape

(101,)

In [5]:
# tokenize the inputs
tokenizer_inputs = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer_inputs.fit_on_texts(input_texts)
input_sequences = tokenizer_inputs.texts_to_sequences(input_texts)

# get the word to index mapping for input language
word2idx_inputs = tokenizer_inputs.word_index
print('Found %s unique input tokens.' % len(word2idx_inputs))

# determine maximum length input sequence
max_len_input = max(len(s) for s in input_sequences)

# tokenize the outputs
# don't filter out special characters
# otherwise <sos> and <eos> won't appear
tokenizer_outputs = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
tokenizer_outputs.fit_on_texts(target_texts + target_texts_inputs) # inefficient, oh well
target_sequences = tokenizer_outputs.texts_to_sequences(target_texts)
target_sequences_inputs = tokenizer_outputs.texts_to_sequences(target_texts_inputs)

# get the word to index mapping for output language
word2idx_outputs = tokenizer_outputs.word_index
print('Found %s unique output tokens.' % len(word2idx_outputs))

# store number of output words for later
# remember to add 1 since indexing starts at 1
num_words_output = len(word2idx_outputs) + 1

# determine maximum length output sequence
max_len_target = max(len(s) for s in target_sequences)

print("max_len_target {}".format(max_len_target))

Found 56 unique input tokens.
Found 131 unique output tokens.
max_len_target 4


In [6]:
# pad the sequences
encoder_inputs = pad_sequences(input_sequences, maxlen=max_len_input)
print("encoder_data.shape:", encoder_inputs.shape)
print("encoder_data[0]:", encoder_inputs[0])

decoder_inputs = pad_sequences(target_sequences_inputs, maxlen=max_len_target, padding='post')
print("decoder_data[0]:", decoder_inputs[0])
print("decoder_data.shape:", decoder_inputs.shape)

decoder_targets = pad_sequences(target_sequences, maxlen=max_len_target, padding='post')

encoder_data.shape: (101, 2)
encoder_data[0]: [0 1]
decoder_data[0]: [ 2 22  0  0]
decoder_data.shape: (101, 4)


In [20]:
max_len_input,max_len_target

(2, 4)

In [21]:
# store all the pre-trained word vectors
print('Loading word vectors...')
count = 0
word2vec = {}
with open(os.path.join('./large_files/glove.6B/glove.6B.%sd.txt' % EMBEDDING_DIM)) as f:
  # is just a space-separated text file in the format:
  # word vec[0] vec[1] vec[2] ...
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
        count += 1
        if count > 100:
            break
print('Found %s word vectors.' % len(word2vec))




# prepare embedding matrix
print('Filling pre-trained embeddings...')
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx_inputs.items():
    if i < MAX_NUM_WORDS:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all zeros.
            embedding_matrix[i] = embedding_vector



Loading word vectors...
Found 101 word vectors.
Filling pre-trained embeddings...


In [23]:
num_words,EMBEDDING_DIM,max_len_input

(57, 100, 2)

In [24]:
# create embedding layer
embedding_layer = Embedding(
  num_words,
  EMBEDDING_DIM,
  weights=[embedding_matrix],
  input_length=max_len_input,
  # trainable=True
)
embedding_layer

In [25]:
num_words_output

132

In [26]:
# create targets, since we cannot use sparse
# categorical cross entropy when we have sequences
decoder_targets_one_hot = np.zeros(
  (
    len(input_texts),
    max_len_target,
    num_words_output
  ),
  dtype='float32'
)

# assign the values
for i, d in enumerate(decoder_targets):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1


In [28]:
decoder_targets_one_hot.shape,decoder_targets.shape

((101, 4, 132), (101, 4))

In [31]:
LATENT_DIM,EMBEDDING_DIM,num_words_output

(256, 100, 132)

In [40]:
# Set up the encoder - simple!
encoder_inputs_placeholder = Input(shape=(max_len_input,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = Bidirectional(LSTM(
  LATENT_DIM,
  return_sequences=True,
  # dropout=0.5 # dropout not available on gpu
))
encoder_outputs = encoder(x)


# Set up the decoder - not so simple
decoder_inputs_placeholder = Input(shape=(max_len_target,))

# this word embedding will not use pre-trained vectors
# although you could
decoder_embedding = Embedding(num_words_output, EMBEDDING_DIM)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)


In [16]:
max_len_input,LATENT_DIM,encoder_outputs,max_len_target,decoder_inputs_x

(2,
 256,
 <tf.Tensor 'bidirectional_1/concat:0' shape=(?, ?, 512) dtype=float32>,
 4,
 <tf.Tensor 'embedding_2/GatherV2:0' shape=(?, 4, 100) dtype=float32>)

encoder_outputs =$N \cdot T_x \cdot 2M_1$  
decoder_inputs_x = $N \cdot T_y \cdot embedding\ dim(100)$

- z = concat[ s(t-1),$h(t')$] 
- z = $tanh(W_1 z + b_1)$ # layer 1 of ANN
- z = softmax($W_2 z + b_2)$ # layer 2 of ANN
- Alphas must sum to 1 over time
$\sum_{t'=1}^{T_x} \alpha(t') = 1 $
- we use this special softmax over time
\begin{equation*}
\alpha(t') = \frac{exp(out(t'))}{\sum_{\tau=1}^{T_x} exp(out(\tau))}
\end{equation*}

In [37]:

######### Attention #########
# Attention layers need to be global because
# they will be repeated Ty times at the decoder
attn_repeat_layer = RepeatVector(max_len_input)
attn_concat_layer = Concatenate(axis=-1)
attn_dense1 = Dense(10, activation='tanh')
attn_dense2 = Dense(1, activation=softmax_over_time)
attn_dot = Dot(axes=1) # to perform the weighted sum of alpha[t] * h[t]

def one_step_attention(h, st_1):
    # h = h(1), ..., h(Tx), shape = (Tx, LATENT_DIM * 2)
    # st_1 = s(t-1), shape = (LATENT_DIM_DECODER,)

    # copy s(t-1) Tx times
    # now shape = (Tx, LATENT_DIM_DECODER)
    st_1 = attn_repeat_layer(st_1)
    
    # Concatenate all h(t)'s with s(t-1)
    # Now of shape (Tx, LATENT_DIM_DECODER + LATENT_DIM * 2)
    x = attn_concat_layer([h, st_1])

    # Neural net first layer
    x = attn_dense1(x)

    # Neural net second layer with special softmax over time
    alphas = attn_dense2(x)

    # "Dot" the alphas and the h's
    # Remember a.dot(b) = sum over a[t] * b[t]
    context = attn_dot([alphas, h])

    return context


In [43]:
h = encoder_outputs
st_1 = initial_s
st_1 = attn_repeat_layer(st_1)
x = attn_concat_layer([h, st_1])
x = attn_dense1(x)
alphas = attn_dense2(x)
context = attn_dot([alphas, h])


In [45]:
x,alphas,context,h,st_1,initial_s

(<tf.Tensor 'dense_4/Tanh:0' shape=(?, 2, 10) dtype=float32>,
 <tf.Tensor 'dense_5/truediv:0' shape=(?, 2, 1) dtype=float32>,
 <tf.Tensor 'dot_2/MatMul:0' shape=(?, 1, 512) dtype=float32>,
 <tf.Tensor 'bidirectional_1/concat:0' shape=(?, ?, 512) dtype=float32>,
 <tf.Tensor 'repeat_vector_3/Tile:0' shape=(?, 2, 256) dtype=float32>,
 <tf.Tensor 's0:0' shape=(?, 256) dtype=float32>)

- st_1 = $N \cdot T_x \cdot (LATENT\ DIM\ DECODER)$
- RepeatVector($T_x$) : $T_x$ times of S(t-1)  
- x = attn_concat_layer([h,st_1]) : x = concat([h(1)...h($T_x$),s(t-1)...s(t-1)])  
- x = attn_concat_layer([h, st_1]) : $N \cdot T_x \cdot (2M_x + M_y)$
- x = attn_dense1(x) : $N \cdot T_x \cdot (10: dense layer outputs)$
- alphas = attn_dense2(x) :  $N \cdot T_x \cdot 1(softmax\ output)$
- context = attn_dot([alphas,h]): 
    - $(T_x',1) (T_x',2M_1) -> (1, 2M_1)$
    - $\alpha * h = \sum_{t'}^{T_x} \alpha(t')h(t')$

In [48]:
# define the rest of the decoder (after attention)
decoder_lstm = LSTM(LATENT_DIM_DECODER, return_state=True)
decoder_dense = Dense(num_words_output, activation='softmax')

initial_s = Input(shape=(LATENT_DIM_DECODER,), name='s0')
initial_c = Input(shape=(LATENT_DIM_DECODER,), name='c0')
context_last_word_concat_layer = Concatenate(axis=2)



In [50]:
LATENT_DIM_DECODER,num_words_output,LATENT_DIM,max_len_target

(256, 132, 256, 4)

In [49]:
# Unlike previous seq2seq, we cannot get the output
# all in one step
# Instead we need to do Ty steps
# And in each of those steps, we need to consider
# all Tx h's

# s, c will be re-assigned in each iteration of the loop
s = initial_s
c = initial_c

# collect outputs in a list at first
outputs = []
for t in range(max_len_target): # Ty times
    # get the context using attention
    context = one_step_attention(encoder_outputs, s)

    # we need a different layer for each time step
    selector = Lambda(lambda x: x[:, t:t+1])
    xt = selector(decoder_inputs_x)

    # combine 
    decoder_lstm_input = context_last_word_concat_layer([context, xt])

    # pass the combined [context, last word] into the LSTM
    # along with [s, c]
    # get the new [s, c] and output
    o, s, c = decoder_lstm(decoder_lstm_input, initial_state=[s, c])

    # final dense layer to get next word prediction
    decoder_outputs = decoder_dense(o)
    outputs.append(decoder_outputs)



Tensor("repeat_vector_3_1/Tile:0", shape=(?, 2, 256), dtype=float32)
Tensor("repeat_vector_3_2/Tile:0", shape=(?, 2, 256), dtype=float32)
Tensor("repeat_vector_3_3/Tile:0", shape=(?, 2, 256), dtype=float32)
Tensor("repeat_vector_3_4/Tile:0", shape=(?, 2, 256), dtype=float32)


In [43]:
xt,decoder_lstm_input,decoder_outputs,num_words_output

(<tf.Tensor 'lambda_4/strided_slice:0' shape=(?, 1, 100) dtype=float32>,
 <tf.Tensor 'concatenate_2_3/concat:0' shape=(?, 1, 612) dtype=float32>,
 <tf.Tensor 'dense_3_3/Softmax:0' shape=(?, 132) dtype=float32>,
 132)

In [26]:
encoder_outputs,s,context

(<tf.Tensor 'bidirectional_2/concat:0' shape=(?, ?, 512) dtype=float32>,
 <tf.Tensor 'lstm_3_3/while/Exit_2:0' shape=(?, 256) dtype=float32>,
 <tf.Tensor 'dot_1_3/MatMul:0' shape=(?, 1, 512) dtype=float32>)

In [41]:

# 'outputs' is now a list of length Ty
# each element is of shape (batch size, output vocab size)
# therefore if we simply stack all the outputs into 1 tensor
# it would be of shape T x N x D
# we would like it to be of shape N x T x D

def stack_and_transpose(x):
    # x is a list of length T, each element is a batch_size x output_vocab_size tensor
    x = K.stack(x) # is now T x batch_size x output_vocab_size tensor
    x = K.permute_dimensions(x, pattern=(1, 0, 2)) # is now batch_size x T x output_vocab_size
    return x

# make it a layer
stacker = Lambda(stack_and_transpose)
outputs = stacker(outputs)

# create the model
model = Model(
  inputs=[
    encoder_inputs_placeholder,
    decoder_inputs_placeholder,
    initial_s, 
    initial_c,
  ],
  outputs=outputs
)

# compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# train the model
z = np.zeros((NUM_SAMPLES, LATENT_DIM_DECODER)) # initial [s, c]



In [ ]:
r = model.fit(
  [encoder_inputs, decoder_inputs, z, z], decoder_targets_one_hot,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_split=0.2
)


In [ ]:
# plot some data
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

# accuracies
plt.plot(r.history['acc'], label='acc')
plt.plot(r.history['val_acc'], label='val_acc')
plt.legend()
plt.show()

In [ ]:

##### Make predictions #####
# As with the poetry example, we need to create another model
# that can take in the RNN state and previous word as input
# and accept a T=1 sequence.

# The encoder will be stand-alone
# From this we will get our initial decoder hidden state
# i.e. h(1), ..., h(Tx)
encoder_model = Model(encoder_inputs_placeholder, encoder_outputs)

# next we define a T=1 decoder model
encoder_outputs_as_input = Input(shape=(max_len_input, LATENT_DIM * 2,))
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

# no need to loop over attention steps this time because there is only one step
context = one_step_attention(encoder_outputs_as_input, initial_s)

# combine context with last word
decoder_lstm_input = context_last_word_concat_layer([context, decoder_inputs_single_x])

# lstm and final dense
o, s, c = decoder_lstm(decoder_lstm_input, initial_state=[initial_s, initial_c])
decoder_outputs = decoder_dense(o)


# note: we don't really need the final stack and tranpose
# because there's only 1 output
# it is already of size N x D
# no need to make it 1 x N x D --> N x 1 x D



# create the model object
decoder_model = Model(
  inputs=[
    decoder_inputs_single,
    encoder_outputs_as_input,
    initial_s, 
    initial_c
  ],
  outputs=[decoder_outputs, s, c]
)



# map indexes back into real words
# so we can view the results
idx2word_eng = {v:k for k, v in word2idx_inputs.items()}
idx2word_trans = {v:k for k, v in word2idx_outputs.items()}





def decode_sequence(input_seq):
    # Encode the input as state vectors.
    enc_out = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))

    # Populate the first character of target sequence with the start character.
    # NOTE: tokenizer lower-cases all words
    target_seq[0, 0] = word2idx_outputs['<sos>']

    # if we get this we break
    eos = word2idx_outputs['<eos>']


    # [s, c] will be updated in each loop iteration
    s = np.zeros((1, LATENT_DIM_DECODER))
    c = np.zeros((1, LATENT_DIM_DECODER))


    # Create the translation
    output_sentence = []
    for _ in range(max_len_target):
        o, s, c = decoder_model.predict([target_seq, enc_out, s, c])
        

        # Get next word
        idx = np.argmax(o.flatten())

        # End sentence of EOS
        if eos == idx:
            break

        word = ''
        if idx > 0:
            word = idx2word_trans[idx]
            output_sentence.append(word)

        # Update the decoder input
        # which is just the word just generated
        target_seq[0, 0] = idx

    return ' '.join(output_sentence)






In [ ]:

while True:
    # Do some test translations
    i = np.random.choice(len(input_texts))
    input_seq = encoder_inputs[i:i+1]
    translation = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[i])
    print('Predicted translation:', translation)
    print('Actual translation:', target_texts[i])

    ans = input("Continue? [Y/n]")
    if ans and ans.lower().startswith('n'):
        break


In [45]:
i = 10
input_seq = encoder_inputs[i:i+1]
input_seq,input_seq.shape

(array([[ 0, 31]], dtype=int32), (1, 2))

In [50]:
input_texts[10],target_texts[10]

('Who?', '¿Quién? <eos>')